### Decorators Application (Logger, Stacked Decorators)

> Aaron's Experiments with Decorators Application (`Logger`, `Stacked Decorators`)

In this example, we are going to create a utility decorator that will
> - **log function calls**
>   - *to console, but in practice you would be writing you logs to a file*
>   - e.g. using Python's built-in logger
> - **to a database, etc**     

In [90]:
from datetime import datetime, timezone
print(datetime.now(tz=timezone.utc).isoformat())
print(datetime.now(timezone.utc))

2025-02-21T02:37:13.423042+00:00
2025-02-21 02:37:13.423109+00:00


In [91]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'{fn.__name__}: called {run_dt}')
        return result
    return inner

In [92]:
@logged
def func_1():
    pass

In [93]:
@logged
def func_2():
    pass

In [94]:
func_1()

func_1: called 2025-02-21 02:37:20.998604+00:00


In [95]:
func_2()

func_2: called 2025-02-21 02:37:22.005113+00:00


Now we may additionaly also want to time the function.

We can certianly include the code to do so in our `logged` decorator, 

but we could also just use the @timed decorator we already wrote by stacking our decorators


In [96]:
def timed(fn):
    from functools import wraps
    from time import perf_counter

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print(f'{fn.__name__} ran for {end - start:.6f}s')
        return result

    return inner
    

In [97]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

In [98]:
factorial(10)

factorial: called 2025-02-21 02:37:35.696096+00:00
factorial ran for 0.000056s


3628800

#### 稍微解釋一下 reduce 在做什麼：
> - reduce 是 functools 中的一個函數，用來**將一個序列，例如list或者range，縮減為單一值**
>   - 取出 list 的前兩個 elements，應用指定的函數，如 乘法 mul
>   - 用計算結果和下一個元素繼續應用該函數
>   - 直到整個序列處理完，產生最終結果
>
基本語法：
``` python
from functools import reduce

reduce(function, iterable, initializer=None)
```

> - function：一個接受兩個參數的函數（如 lambda x, y: x + y）。
> - iterable：要被縮減的序列（如 range(1, n+1)）。
> - initializer（可選）：初始值，會作為 function 的第一個參數。

In [99]:
# 在 factorial(n) 這個函數中，我們用 reduce 來計算 n 的階乘

from operator import mul
from functools import reduce

def factorial(n):
    return reduce(mul, range(1, n+1))
    

In [100]:
reduce(mul, [1, 2, 3, 4, 5])

120

In [101]:
factorial(10)

3628800

In [106]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print(f'timed:  {fn.__name__} ran for {end-start:.6f}s')
        return result

    return inner

In [107]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'logged: {fn.__name__} called at: {run_dt.isoformat()}')
        return result
    
    return inner
    

In [108]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

In [109]:
factorial(10)

logged: factorial called at: 2025-02-21T02:38:55.602853+00:00
timed:  factorial ran for 0.000031s


3628800

In [110]:
@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

In [111]:
factorial(10)

timed:  factorial ran for 0.000026s
logged: factorial called at: 2025-02-21T02:39:01.399513+00:00


3628800

#### <font color=lightseagreen> Note that the order in which we stacked decorators essentially can make a difference!

##### Remember that this is because our stacked decorators essentailly amounted to:

In [112]:
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

factorial = timed(logged(factorial))

So in this case the `timed` decorator will be called first, followed by the `logged` decorator.

You may wonder why the printed output seems reversed. Look at how the decorators were defined - they first ran the function passed in, and **then** printed the result.

So in the above example, a simplified look at what happens in each decorator:

* `timed(fn)(*args, **kwargs)`:
    1. calls `fn(*args, **kwargs)`
    2. prints timing
    
    
* `logged(fn)(*args, **kwargs)`:
    1. calls `fn(*args, **kwargs)`
    2. prints log info

So, calling
`factorial = timed(logged(factorial))`

is equivalent to:

<pre>
fn = logged(factorial)
factorial = timed(fn)

factorial(n) --> call timed(fn)(n)
             --> call fn(n), then print timing
             --> call logged(original_factorial)(n), then print timing
             --> call original_factorial(n), then log, then print timing
</pre>

So as you can see, the `timed` decorator ran first, but it called the logged decorated function first, then printed the result - hence why the print output seems reversed.

In [113]:
factorial(10)

logged: factorial called at: 2025-02-21T02:39:16.730856+00:00
timed:  factorial ran for 0.000057s


3628800

##### <font color=lightcoral>But in the following case, the `logged` decorator will run fisrt, followed by the `timed` deco

In [39]:
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

    factorial = logged(timed(factorial))

In [114]:
factorial(10)

logged: factorial called at: 2025-02-21T02:39:21.979297+00:00
timed:  factorial ran for 0.000033s


3628800

In [115]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner



def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner


@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial(10)

factorial: called 2025-02-21 02:39:27.159512+00:00
factorial ran for 0.000035s


3628800

In [116]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner



def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner


def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = timed(logged(factorial))

factorial(10)

factorial: called 2025-02-21 02:39:34.860086+00:00
factorial ran for 0.000030s


3628800

In [117]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner



def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner


# def factorial(n):
#     from operator import mul
#     from functools import reduce
    
#     return reduce(mul, range(1, n+1))

# factorial = timed(logged(factorial))

def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = logged(timed(factorial))

factorial(10)

factorial ran for 0.000005s
factorial: called 2025-02-21 02:39:44.262741+00:00


3628800

In [118]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('logged: {0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner



def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('timed {0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner


# def factorial(n):
#     from operator import mul
#     from functools import reduce
    
#     return reduce(mul, range(1, n+1))

# factorial = timed(logged(factorial))

# def factorial(n):
#     from operator import mul
#     from functools import reduce
    
#     return reduce(mul, range(1, n+1))

# factorial = logged(timed(factorial))

@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce
    

    return reduce(mul, range(1, n+1))

factorial(10)

timed factorial ran for 0.000005s
logged: factorial: called 2025-02-21 02:39:48.945788+00:00


3628800

In [119]:
## 我的
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print(f'timed:  {fn.__name__} ran for {end-start:.6f}s')
        return result

    return inner

def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'logged: {fn.__name__} called at: {run_dt.isoformat()}')
        return result
    
    return inner

@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

factorial(10)

logged: factorial called at: 2025-02-21T02:39:59.600211+00:00
timed:  factorial ran for 0.000363s


3628800

In [120]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('logged: {0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner



def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('timed  :{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner


@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial(10)

logged: factorial: called 2025-02-21 02:40:08.117539+00:00
timed  :factorial ran for 0.000148s


3628800

<font color=mediumseagreen> To make this clearer, let's write two very simple decorators as follows:

In [ ]:
def dec_1(fn):
    def inner():
        print('running dec_1')
        return fn  # 這裡錯誤，應該是 return fn（），否則不會執行
    return inner

In [122]:
def dec_2(fn):
    def inner():
        print('running dec_2')
        return fn()
    return inner

In [123]:
@dec_1
@dec_2
def my_func():
    print('running my_func')

In [124]:
my_func()

running dec_1


<function __main__.dec_2.<locals>.inner()>

In [125]:
def dec_1(fn):
    def inner():
        print('running dec_1')
        return fn()
    return inner

def dec_2(fn):
    def inner():
        print('running dec_2')
        return fn()
    return inner

@dec_1
@dec_2
def my_func():
    print('running my_func')
    
my_func()

running dec_1
running dec_2
running my_func


<font color=crimson> But if we change the order of deco

In [126]:
@dec_2
@dec_1
def my_func():
    print('running my_func')

In [127]:
my_func()

running dec_2
running dec_1
running my_func


In [ ]:
def aaa(fn):
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        print(f'aaa: {fn.__name__}')
        return fn()    
    return inner



def bbb(fn):
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        print(f'bbb: {fn.__name__}')  # 錯誤：這裡沒有return，ccc根本不會執行！！！
    
    return inner


@aaa
@bbb
def my_ccc():
    print('running my_ccc')

my_ccc()

aaa: my_ccc
bbb: my_ccc


### Aaron's Scratch Pad

In [2]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'{run_dt}: called {fn.__name__}')
        return result

    return inner

In [3]:
@logged
def func_1():
    pass

In [5]:
@logged
def func_2():
    pass

In [6]:
func_1()

2025-03-11 04:31:21.596423+00:00: called func_1


In [7]:
func_2()

2025-03-11 04:31:27.297488+00:00: called func_2


In this example we're going to create a utility decorator that will log function calls (to the console, but in practice you would be writing your logs to a file (e.g. using Python's built-in logger), or to a database, etc.

In [1]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner

In [2]:
@logged
def func_1():
    pass

In [3]:
@logged
def func_2():
    pass

In [4]:
func_1()

func_1: called 2017-12-10 00:09:19.443657+00:00


In [5]:
func_2()

func_2: called 2017-12-10 00:09:19.460691+00:00


Now we may additionaly also want to time the function. We can certainly include the code to do so in our `logged` decorator, but we could also just use the `@timed` decorator we already wrote by **stacking** our decorators.

In [6]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner

In [7]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [8]:
factorial(10)

factorial: called 2017-12-10 00:09:19.496762+00:00
factorial ran for 0.000130s


3628800

Note that the order in which we stack the decorators can make a difference!

Remember that this is because our stacked decorators essentially amounted to:

In [9]:
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = timed(logged(factorial))

So in this case the `timed` decorator will be called first, followed by the `logged` decorator.

You may wonder why the printed output seems reversed. Look at how the decorators were defined - they first ran the function passed in, and **then** printed the result.

So in the above example, a simplified look at what happens in each decorator:

* `timed(fn)(*args, **kwargs)`:
    1. calls `fn(*args, **kwargs)`
    2. prints timing
    
    
* `logged(fn)(*args, **kwargs)`:
    1. calls `fn(*args, **kwargs)`
    2. prints log info

So, calling
`factorial = timed(logged(factorial))`

is equivalent to:

<pre>
fn = logged(factorial)
factorial = timed(fn)

factorial(n) --> call timed(fn)(n)
             --> call fn(n), then print timing
             --> call logged(original_factorial)(n), then print timing
             --> call original_factorial(n), then log, then print timing
</pre>

So as you can see, the `timed` decorator ran first, but it called the logged decorated function first, then printed the result - hence why the print output seems reversed.

In [10]:
factorial(10)

factorial: called 2017-12-10 00:09:19.525820+00:00
factorial ran for 0.000147s


3628800

But in the following case, the `logged` decorator will run first, followed by the `timed` decorator:

In [11]:
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = logged(timed(factorial))

In [12]:
factorial(10)

factorial ran for 0.000015s
factorial: called 2017-12-10 00:09:19.547866+00:00


3628800

Or, using the **@** notation:

In [13]:
@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [14]:
factorial(10)

factorial ran for 0.000016s
factorial: called 2017-12-10 00:09:19.572914+00:00


3628800

In [15]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [16]:
factorial(10)

factorial: called 2017-12-10 00:09:19.608237+00:00
factorial ran for 0.000153s


3628800

To make this clearer, let's write two very simple decorators as follows:

In [17]:
def dec_1(fn):
    def inner():
        print('running dec_1')
        return fn()
    return inner

In [18]:
def dec_2(fn):
    def inner():
        print('running dec_2')
        return fn()
    return inner

In [19]:
@dec_1
@dec_2
def my_func():
    print('running my_func')

In [20]:
my_func()

running dec_1
running dec_2
running my_func


But if we change the order of the decorators:

In [21]:
@dec_2
@dec_1
def my_func():
    print('running my_func')

In [22]:
my_func()

running dec_2
running dec_1
running my_func


You may wonder whether this really matters in practice. And yes, it can.

Consider an API that contains various functions that can be called. However, endpoints are secured and can only be run by authenticated users who have some specific role(s). If they do not have the role you want to return an unauthorized error. But if they do, then you want to log that they called the endpoint.

In this case you may have one decorator that is used to check authentication and permissions (and immediately return an unauthorized error from the API if applicable), and the other to log the call. 

If you decorated it this way:

<pre>
@log
@authorize
def my_endpoint():
    pass
</pre>

then the call would always be logged.

But, in this instance:

<pre>
@authorize
@log
def my_endpoint():
    pass
</pre>

your endpoint would only get logged if the user passed the `authorize` test.